# Main Analysis

This file does not rely on user input. So we would like to keep it interchangeable with the actual main_analysis.py file.
So please keep all the >relevant< code within one code block. (If you want to inspect certain elements or do other things that are not relevant to the main analysis, you can of course add them in new blocks).

In [ ]:
import sys
import os.path as op
module_path = op.abspath(op.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import pandas as pd
import numpy as np
import mne

from data_analysis.functions_preprocessing import \
    (combine_raws, split_raws, preprocess_single_sub, mark_bads_and_save,
     run_ica_and_save)
from data_analysis.functions_behavioral import \
    (create_event_df, remove_ghost_triggers, calculate_alpha,
     join_event_dfs, remove_outliers, events_from_event_df)
from data_analysis.functions_connectivity import \
    epochs_ispc
from data_analysis.functions_graph_theory import \
    epochs_weighted_small_world_coeff



subject_dir = "/net/store/nbp/projects/hyperscanning/hyperscanning-2.0/mne_data/sourcedata/"
behav_dir = "/net/store/nbp/projects/hyperscanning/study_project/NBP_Hyperscanning/data_analysis/Behavioural_Analysis/BehaviouralData"



# Main Data Analysis ###################################
# initialize containers to analyze later
connectivity_matrices = []
small_world_coeffs = []

# Perform the data analysis
for subj_pair in ['204']:  #['202','203','204','205','206','207','208','209','211','212']:

    subs_path = subject_dir + "sub-{0}/eeg/sub-{0}_task-hyper_eeg.fif".format(subj_pair)
    behav_path = op.join(behav_dir, "{0}.csv".format(subj_pair))

    combined_raw = mne.io.read_raw_fif(subs_path, preload=True)

    # split the subjects and delete the raw file
    raws = split_raws(combined_raw)
    del combined_raw

    # apply the preprocessing
    #for idx, raw in enumerate(raws):
    #    subj_idx = "sub-{0}_p-{1}".format(subj_pair, idx)
    #    raws[idx] = preprocess_single_sub(raw, subj_idx)


    # combine the subjects again
    raw_combined = combine_raws(raws[0], raws[1])
    del raws  # to save memory

    # do the behavioral analysis and get the epochs
    behavioral_df = calculate_alpha(pd.read_csv(behav_path))
    
    event_df = create_event_df(raw_combined)
    event_df = remove_ghost_triggers(event_df)
    event_df = join_event_dfs(event_df, behavioral_df)
    event_df = remove_outliers(event_df, exclude_stddev=2)
    events = events_from_event_df(event_df)

    # define the parameters for epoching
    # TODO: Define events more elaborate!
    event_id = 11
    tmin = 0
    tmax = 1

    # epoch the data. Here we filter out bad segments from both participants
    # TODO: do we need a baseline for the connectivity analysis?
    epochs = mne.Epochs(raw_combined, events, event_id, tmin, tmax,
                        picks=["eeg"], baseline=(0, 0),
                        reject_by_annotation=True)[:10] # only use the first ten epochs
    
    # calculate the ISPC
    freqs = np.arange(5,15,5)
    cycles = freqs / 2.
    ispc_matrix, freqs, times = epochs_ispc(epochs, freqs, cycles, n_jobs=4)
    
    print("ISPC DONE")
    
    # calculate the small world coefficient
    small_worlds = epochs_weighted_small_world_coeff(ispc_matrix)
    
    # append the results to the respective lists
    connectivity_matrices.append(ispc_matrix)
    small_world_coeffs.append(small_worlds)



Opening raw data file /net/store/nbp/projects/hyperscanning/hyperscanning-2.0/mne_data/sourcedata/sub-204/eeg/sub-204_task-hyper_eeg.fif...


<ipython-input-2-ad92dac54aea>:40: RuntimeWarning: This filename (/net/store/nbp/projects/hyperscanning/hyperscanning-2.0/mne_data/sourcedata/sub-204/eeg/sub-204_task-hyper_eeg.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz or _meg.fif
  combined_raw = mne.io.read_raw_fif(subs_path, preload=True)


Isotrak not found
    Range : 0 ... 3537048 =      0.000 ...  3454.148 secs
Ready.
Reading 0 ... 3537048  =      0.000 ...  3454.148 secs...
6638 events found
Event IDs: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 36 37 38 39 40 41 42 43 44 45 46 48 49]
267 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
Loading data for 10 events and 1025 original time points ...
0 bad epochs dropped


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  14 tasks      | elapsed:    3.2s


Not setting metadata


[Parallel(n_jobs=4)]: Done 112 tasks      | elapsed:    4.1s
[Parallel(n_jobs=4)]: Done 128 out of 128 | elapsed:    4.3s finished
/net/store/nbp/projects/hyperscanning/study_project/dirk/NBP_Hyperscanning/data_analysis/functions_connectivity.py:44: ComplexWarning: Casting complex values to real discards the imaginary part
  ispc_matrix[epoch, chan_a, chan_b, freq] = core_ispc(phase_diff[freq], times)


In [ ]:
# NOTE: There seems to be a bug in the data from the very beginning.
# The channels are somehow already referenced, even though we don't have a ref.
# and the one thats supposed to be mapped to Cz (for subject 2) is not the one thats referenced.
# Instead, the Oz electrode was used for the reference. Looks like we have to dig in the script from Max.
# Or change the mapping order.
print(raw_combined.get_data(picks=["sub1_Cz", "sub2_Oz"]))

## Investigate stuff

Here, you can look into all the stuff, connectivity matrices etc.

In [ ]:
#event_df[event_df["alpha"] > 360]
event_df

In [ ]:
# plot the ISPC matrix
from functions_connectivity import plot_connectivity_matrix

print("Freqs: ", freqs)

plot_connectivity_matrix(np.squeeze(ispc_matrix[0, :,:,0]), epochs.ch_names, title="ISPC for freq = "+ str(freqs[0]))

plot_connectivity_matrix(np.squeeze(ispc_matrix[0, :,:,1]), epochs.ch_names, title="ISPC for freq = "+ str(freqs[1]))

In [ ]:
# plot the average ISPC matrix
avg_ispc = np.mean(ispc_matrix, axis=0)

plot_connectivity_matrix(np.squeeze(avg_ispc[:,:,0]), epochs.ch_names, title="ISPC for freq = "+ str(freqs[0]))
plot_connectivity_matrix(np.squeeze(avg_ispc[:,:,1]), epochs.ch_names, title="ISPC for freq = "+ str(freqs[1]))

In [ ]:
# plot a connectivity circle
from mne.viz import plot_connectivity_circle

mne.viz.plot_connectivity_circle(ispc_matrix[0, :,:,1], epochs.ch_names, vmin=0.8)

In [ ]:
# plot a connectivity circle for the average matrix

mne.viz.plot_connectivity_circle(avg_ispc[:,:,1], epochs.ch_names)

In [ ]:
print(np.where(np.array(epochs.ch_names) == "sub1_Cz"),
      np.where(np.array(epochs.ch_names) == "sub2_Oz"))

ispc_matrix[:, 15, 94, 1]
epochs.get_data()[:,15,:]
epochs.get_data()[:,94,:]